In [3]:
import  cerberus
import glob
import pyranges as pr
import pandas as pd

In [4]:
df = pd.DataFrame()
for f in glob.glob('*gtf'):
    print(f)
    df = pd.concat([pr.read_gtf(f, as_df=True), df], axis=0)

GM10493_1.gtf
GM12878_1.gtf


In [5]:
temp = df.loc[df.Feature=='transcript']
temp.loc[temp.transcript_id.duplicated(keep=False)].sort_values(by='transcript_id')[['gene_id', 'transcript_id']]

,gene_id,transcript_id
140848,ENSG00000000419.14,"ENSG00000000419[1,1,1]"
133260,ENSG00000000419.14,"ENSG00000000419[1,1,1]"
140868,ENSG00000000419.14,"ENSG00000000419[1,14,1]"
133280,ENSG00000000419.14,"ENSG00000000419[1,14,1]"
133270,ENSG00000000419.14,"ENSG00000000419[1,16,1]"
...,...,...
200267,ENSG00000310458.1,"ENSG00000310458[1,3,1]"
209601,ENSG00000310521.1,"ENSG00000310521[2,10,3]"
200696,ENSG00000310521.1,"ENSG00000310521[2,10,3]"
84178,ENSG00000310525.1,"ENSG00000310525[2,3,2]"


In [6]:
len(df.loc[df.gene_id.str.contains('novel_gene')].index)

12606

In [7]:
len(df.loc[(df.gene_id.str.contains('novel_gene'))&\
           (df.transcript_id.isnull())&\
           (df.Feature=='transcript')].index)

0

In [8]:
# df.loc[df.gene_id.str.contains('novel_gene'), 'Chromosome'].unique().tolist()

In [9]:
# all of these problems are from novel genes, 
# and all novel genes are problems

# the novel bois didnt get added to h5

In [10]:
temp = df.loc[df.Feature=='transcript']
temp.loc[temp.transcript_id.duplicated(keep=False)].sort_values(by='transcript_id')

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,exons,tss_first_sd_issue,original_transcript_id,tes_last_sa_issue,transcript_support_level,protein_id,havana_transcript,ccdsid,havana_gene,hgnc_id
140848,chr20,HAVANA,transcript,50934856,50958580,.,-,.,ENSG00000000419.14,protein_coding,...,9,False,ENST00000371588.10,False,1,ENSP00000360644.5,OTTHUMT00000079716.1,CCDS13434.1,OTTHUMG00000032742.2,HGNC:3005
133260,chr20,HAVANA,transcript,50934856,50958580,.,-,.,ENSG00000000419.14,protein_coding,...,9,False,ENST00000371588.10,False,1,ENSP00000360644.5,OTTHUMT00000079716.1,CCDS13434.1,OTTHUMG00000032742.2,HGNC:3005
140868,chr20,HAVANA,transcript,50934856,50958580,.,-,.,ENSG00000000419.14,protein_coding,...,7,False,ENST00000683048.1,False,NaN,ENSP00000506986.1,NaN,NaN,OTTHUMG00000032742.2,HGNC:3005
133280,chr20,HAVANA,transcript,50934856,50958580,.,-,.,ENSG00000000419.14,protein_coding,...,7,False,ENST00000683048.1,False,NaN,ENSP00000506986.1,NaN,NaN,OTTHUMG00000032742.2,HGNC:3005
133270,chr20,HAVANA,transcript,50934856,50958580,.,-,.,ENSG00000000419.14,protein_coding,...,9,False,ENST00000466152.5,False,1,ENSP00000507119.1,OTTHUMT00000079717.1,NaN,OTTHUMG00000032742.2,HGNC:3005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200267,chr7,HAVANA,transcript,44194249,44198255,.,+,.,ENSG00000310458.1,lncRNA,...,2,False,ENST00000849948.1,False,NaN,NaN,NaN,NaN,NaN,NaN
209601,chr8,HAVANA,transcript,119855428,119869023,.,+,.,ENSG00000310521.1,lncRNA,...,2,False,ENST00000765898.1,False,NaN,NaN,NaN,NaN,OTTHUMG00000195600.1,NaN
200696,chr8,HAVANA,transcript,119855428,119869023,.,+,.,ENSG00000310521.1,lncRNA,...,2,False,ENST00000765898.1,False,NaN,NaN,NaN,NaN,OTTHUMG00000195600.1,NaN
84178,chr16,NaN,transcript,72084845,72093681,.,-,.,ENSG00000310525.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df = cerberus.sort_gtf(df)

In [16]:
ca = cerberus.read('../ref/ref.h5')

tss = ca.tss
tss.rename({'Name': 'tss_id'}, axis=1, inplace=True)
tes = ca.tes
tes.rename({'Name': 'tes_id'}, axis=1, inplace=True)

tss = pr.PyRanges(tss)
tes = pr.PyRanges(tes)

df = cerberus.update_gtf_ends(df, tss, tes)
tss.head()

,Chromosome,Start,End,Strand,tss_id,source,novelty,gene_id,tss
0,GL000008.2,83306,83446,+,ENSG00000296732_1,"47,GM12878_1",Known,ENSG00000296732,1
1,GL000008.2,85532,85583,+,ENSG00000296732_2,47,Known,ENSG00000296732,2
2,GL000008.2,208519,208637,-,ENSG00000296756_1,47,Known,ENSG00000296756,1
3,GL000008.2,170299,170454,-,ENSG00000296775_1,"47,GM10493_1,GM12878_1",Known,ENSG00000296775,1
4,GL000009.2,35368,35419,+,ENSG00000296902_1,47,Known,ENSG00000296902,1
5,GL000009.2,5095,5222,+,ENSG00000297619_1,47,Known,ENSG00000297619,1
6,GL000009.2,82811,82862,+,ENSG00000306744_1,47,Known,ENSG00000306744,1
7,GL000009.2,43268,43322,+,ENSG00000306767_1,47,Known,ENSG00000306767,1


In [17]:
df.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,exons,tss_first_sd_issue,original_transcript_id,tes_last_sa_issue,transcript_support_level,protein_id,havana_transcript,ccdsid,havana_gene,hgnc_id
0,chr1,NaN,gene,169794786,169853095,.,+,.,ENSG00000000460.17,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,NaN,gene,169794786,169853095,.,+,.,ENSG00000000460.17,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,NaN,transcript,169795014,169804396,.,+,.,ENSG00000000460.17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,NaN,exon,169795014,169795213,.,+,.,ENSG00000000460.17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,NaN,exon,169798917,169798958,.,+,.,ENSG00000000460.17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
print(len(df.index))
df = cerberus.agg_gtf(df)
print(len(df.index))

447907


/Users/fairliereese/Documents/programming/mortazavi_lab/bin/cerberus/cerberus/cerberus.py:1184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_df[non_gb_cols].fillna('', inplace=True)
/Users/fairliereese/Documents/programming/mortazavi_lab/bin/cerberus/cerberus/cerberus.py:1195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(non_gb_cols, axis=1, inplace=True)


447892


In [19]:
temp = df.loc[df.Feature=='gene']
temp.loc[temp.gene_id.duplicated(keep=False)].sort_values(by='gene_id')

,Chromosome,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,gene_name,...,tss_first_sd_issue,original_transcript_name,transcript_support_level,ccdsid,havana_transcript,tag,havana_gene,protein_id,tes_last_sa_issue,hgnc_id
221622,chr20,gene,50934856,50958580,.,-,.,ENSG00000000419.14,protein_coding,ENSG00000000419.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221621,chr20,gene,50934856,50958580,.,-,.,ENSG00000000419.14,protein_coding,ENSG00000000419.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221724,chr1,gene,169849620,169893984,.,-,.,ENSG00000000457.14,protein_coding,ENSG00000000457.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221723,chr1,gene,169849620,169893984,.,-,.,ENSG00000000457.14,protein_coding,ENSG00000000457.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,chr1,gene,169794786,169853095,.,+,.,ENSG00000000460.17,protein_coding,ENSG00000000460.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447130,chr7,gene,716607,727719,.,-,.,novel_gene_chr7_59,NaN,novel_gene_chr7_59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447362,chr9,gene,37400497,37402888,.,-,.,novel_gene_chr9_1367,NaN,novel_gene_chr9_1367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221110,chr9,gene,37407261,37410166,.,+,.,novel_gene_chr9_1367,NaN,novel_gene_chr9_1367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221227,chr9,gene,128618891,128624727,.,+,.,novel_gene_chr9_3435,NaN,novel_gene_chr9_3435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# duplicated genes?
gb_cols = ['Chromosome',
             'Feature',
             'Start', 'End',
             'Score', 'Strand', 'Frame', 'gene_id', 'gene_name',
             'gene_status', 'gene_type', 'talon_gene',
             'ic', 'ic_id', 'tss_id', 'tss', 'tes_id', 'tes', 'transcript_id',
             'transcript_name']
gb_cols = list(set(df.columns)&set(gb_cols))
temp = df.loc[df.Feature=='gene']
temp.loc[temp.gene_id.duplicated(keep=False)].sort_values(by='gene_id')[gb_cols].head()
# assert len(temp.loc[temp.gene_id.duplicated()].index) == 0

,transcript_id,Score,gene_type,Feature,gene_id,tes_id,tss_id,tss,End,Strand,transcript_name,tes,Frame,Start,Chromosome,ic_id,gene_name,ic
221622,NaN,.,protein_coding,gene,ENSG00000000419.14,NaN,NaN,NaN,50958580,-,NaN,NaN,.,50934856,chr20,NaN,ENSG00000000419.14,NaN
221621,NaN,.,protein_coding,gene,ENSG00000000419.14,NaN,NaN,NaN,50958580,-,NaN,NaN,.,50934856,chr20,NaN,ENSG00000000419.14,NaN
221724,NaN,.,protein_coding,gene,ENSG00000000457.14,NaN,NaN,NaN,169893984,-,NaN,NaN,.,169849620,chr1,NaN,ENSG00000000457.14,NaN
221723,NaN,.,protein_coding,gene,ENSG00000000457.14,NaN,NaN,NaN,169893984,-,NaN,NaN,.,169849620,chr1,NaN,ENSG00000000457.14,NaN
0,NaN,.,protein_coding,gene,ENSG00000000460.17,NaN,NaN,NaN,169853095,+,NaN,NaN,.,169794786,chr1,NaN,ENSG00000000460.17,NaN


In [21]:
# duplicated transcripts?
gb_cols = ['Chromosome',
             'Feature',
             'Start', 'End',
             'Score', 'Strand', 'Frame', 'gene_id', 'gene_name',
             'gene_status', 'gene_type', 'talon_gene',
             'ic', 'ic_id', 'tss_id', 'tss', 'tes_id', 'tes', 'transcript_id',
             'transcript_name']
gb_cols = list(set(df.columns)&set(gb_cols))
temp = df.loc[df.Feature=='transcript']
temp.loc[temp.transcript_id.duplicated(keep=False)].sort_values(by='transcript_id')[gb_cols].head()
# assert len(temp.loc[temp.gene_id.duplicated()].index) == 0

,transcript_id,Score,gene_type,Feature,gene_id,tes_id,tss_id,tss,End,Strand,transcript_name,tes,Frame,Start,Chromosome,ic_id,gene_name,ic
221624,"ENSG00000000419[1,1,1]",.,protein_coding,transcript,ENSG00000000419.14,ENSG00000000419_1,ENSG00000000419_1,1.0,50958580,-,"ENSG00000000419.14[1,1,1]",1.0,.,50934856,chr20,ENSG00000000419_1,ENSG00000000419.14,1.0
221623,"ENSG00000000419[1,1,1]",.,protein_coding,transcript,ENSG00000000419.14,ENSG00000000419_1,ENSG00000000419_1,1.0,50958580,-,"ENSG00000000419.14[1,1,1]",1.0,.,50934856,chr20,ENSG00000000419_1,ENSG00000000419.14,1.0
221644,"ENSG00000000419[1,14,1]",.,protein_coding,transcript,ENSG00000000419.14,ENSG00000000419_1,ENSG00000000419_1,1.0,50958580,-,"ENSG00000000419.14[1,14,1]",1.0,.,50934856,chr20,ENSG00000000419_14,ENSG00000000419.14,14.0
221643,"ENSG00000000419[1,14,1]",.,protein_coding,transcript,ENSG00000000419.14,ENSG00000000419_1,ENSG00000000419_1,1.0,50958580,-,"ENSG00000000419.14[1,14,1]",1.0,.,50934856,chr20,ENSG00000000419_14,ENSG00000000419.14,14.0
221660,"ENSG00000000419[1,16,1]",.,protein_coding,transcript,ENSG00000000419.14,ENSG00000000419_1,ENSG00000000419_1,1.0,50958580,-,"ENSG00000000419.14[1,16,1]",1.0,.,50934856,chr20,ENSG00000000419_16,ENSG00000000419.14,16.0


In [24]:
print(len(temp.transcript_id.unique()))
print(len(temp.index))
print(len(temp[gb_cols].drop_duplicates().index))

42862
57625
42862


In [43]:
# duplicated transcripts?
temp = df.loc[df.Feature=='transcript']
temp.loc[temp.transcript_id.duplicated(keep=False)].sort_values(by='transcript_id')[gb_cols].head()
# assert len(temp.loc[temp.transcript_id.duplicated()].index) == 0

,Chromosome,transcript_name,ic_id,ic,Feature,tes_id,Start,transcript_id,Score,tes,gene_type,End,tss,tss_id,Strand,gene_name,gene_id,Frame
221624,chr20,"ENSG00000000419.14[1,1,1]",ENSG00000000419_1,1.0,transcript,ENSG00000000419_1,50934856,"ENSG00000000419[1,1,1]",.,1.0,protein_coding,50958580,1.0,ENSG00000000419_1,-,ENSG00000000419.14,ENSG00000000419.14,.
221623,chr20,"ENSG00000000419.14[1,1,1]",ENSG00000000419_1,1.0,transcript,ENSG00000000419_1,50934856,"ENSG00000000419[1,1,1]",.,1.0,protein_coding,50958580,1.0,ENSG00000000419_1,-,ENSG00000000419.14,ENSG00000000419.14,.
221644,chr20,"ENSG00000000419.14[1,14,1]",ENSG00000000419_14,14.0,transcript,ENSG00000000419_1,50934856,"ENSG00000000419[1,14,1]",.,1.0,protein_coding,50958580,1.0,ENSG00000000419_1,-,ENSG00000000419.14,ENSG00000000419.14,.
221643,chr20,"ENSG00000000419.14[1,14,1]",ENSG00000000419_14,14.0,transcript,ENSG00000000419_1,50934856,"ENSG00000000419[1,14,1]",.,1.0,protein_coding,50958580,1.0,ENSG00000000419_1,-,ENSG00000000419.14,ENSG00000000419.14,.
221660,chr20,"ENSG00000000419.14[1,16,1]",ENSG00000000419_16,16.0,transcript,ENSG00000000419_1,50934856,"ENSG00000000419[1,16,1]",.,1.0,protein_coding,50958580,1.0,ENSG00000000419_1,-,ENSG00000000419.14,ENSG00000000419.14,.


In [25]:
len(df.loc[df.duplicated(subset=gb_cols)])
df = df.drop_duplicates(subset=gb_cols, keep='first')
df = cerberus.sort_gtf(df)
df = pr.PyRanges(df)
df.to_gtf('test_merged.gtf')

In [29]:
df = cerberus.sort_gtf(df)
df = pr.PyRanges(df)
df.to_gtf('test_merged.gtf')

In [31]:
df = df.df
df = df.loc[df.Feature.isin(['transcript', 'exon'])]
df = pr.PyRanges(df)
df.to_gtf('test_merged_no_genes.gtf')

In [27]:
df = df.df
len(df.loc[df.Feature=='transcript'].index)

42862

In [18]:
gb_cols = ['Chromosome',
                 'Feature',
                 'Start', 'End',
                 'Score', 'Strand', 'Frame', 'gene_id', 'gene_name',
                 'gene_status', 'gene_type', 'talon_gene',
                 'ic', 'ic_id', 'tss_id', 'tss', 'tes_id', 'tes', 'transcript_id',
                 'transcript_name']
gb_cols = list(set(temp.columns)&set(gb_cols))

In [20]:
temp = temp[gb_cols]
print(len(temp.index))
temp.drop_duplicates(inplace=True)
print(len(temp.index))

67390
44489


In [21]:
def collapse_non_gb_col(x):
    x = x.fillna('')
    x = x.astype(str)
    x = x.unique().tolist()
    x = ','.join(x)
    if x == '':
        x = np.nan
    return(x)

gb_cols = ['Chromosome',
             'Feature',
             'Start', 'End',
             'Score', 'Strand', 'Frame', 'gene_id', 'gene_name',
             'gene_status', 'gene_type', 'talon_gene',
             'ic', 'ic_id', 'tss_id', 'tss', 'tes_id', 'tes', 'transcript_id',
             'transcript_name']
gb_cols = list(set(df.columns)&set(gb_cols))

# get dictionary:function mapping for aggregation
agg_cols = list(set(df.columns)-set(gb_cols))
agg_dict = dict()
for c in agg_cols:
    agg_dict[c] = collapse_non_gb_col

# get collapsed features to add to deduplicated df
t_df = df.loc[df.Feature == 'transcript'].copy(deep=True)

non_gb_cols = list(set(t_df.columns.tolist())-set(gb_cols))
t_df[non_gb_cols].fillna('', inplace=True)
t_df = t_df.groupby(gb_cols, observed=True).agg(agg_dict).reset_index()
t_df = t_df[['transcript_id']+non_gb_cols]
collapsed_feats = t_df.copy(deep=True)

# # replace the non gb columns with the ones that we already grouped
# df.drop(non_gb_cols, axis=1, inplace=True)
# df = df.merge(collapsed_feats, how='left', on='transcript_id'

In [26]:
print(len(t_df.index))
print(len(t_df.transcript_id.unique()))
t_df.head()

42103
42103


,transcript_id,Source,tes_last_sa_issue,original_transcript_name,tss_first_sd_issue,original_transcript_id
0,"ENSG00000000003[1,1,1]",annotated_isoform,False,ENST00000373020.9,False,ENST00000373020.9
1,"ENSG00000000419[1,1,1]",annotated_isoform,False,ENST00000371588.10,False,ENST00000371588.10
2,"ENSG00000000419[1,10,10]",annotated_isoform,False,ENST00000684628.1,False,ENST00000684628.1
3,"ENSG00000000457[1,1,1]",annotated_isoform,False,ENST00000367771.11,False,ENST00000367771.11
4,"ENSG00000000457[3,3,2]",annotated_isoform,False,ENST00000367772.8,False,ENST00000367772.8


In [27]:
print(len(collapsed_feats.index))
print(len(collapsed_feats.transcript_id.unique()))
collapsed_feats.head()

42103
42103


,transcript_id,Source,tes_last_sa_issue,original_transcript_name,tss_first_sd_issue,original_transcript_id
0,"ENSG00000000003[1,1,1]",annotated_isoform,False,ENST00000373020.9,False,ENST00000373020.9
1,"ENSG00000000419[1,1,1]",annotated_isoform,False,ENST00000371588.10,False,ENST00000371588.10
2,"ENSG00000000419[1,10,10]",annotated_isoform,False,ENST00000684628.1,False,ENST00000684628.1
3,"ENSG00000000457[1,1,1]",annotated_isoform,False,ENST00000367771.11,False,ENST00000367771.11
4,"ENSG00000000457[3,3,2]",annotated_isoform,False,ENST00000367772.8,False,ENST00000367772.8


In [28]:
# deduplicate df based only on transcript id
temp = df[['transcript_id', 'original_transcript_id']].drop_duplicates()
print(len(temp.index))
dupe_old_tids = temp.loc[temp.transcript_id.duplicated(keep='first'), 'original_transcript_id']
df = df.loc[~df.original_transcript_id.isin(dupe_old_tids)]


42104


In [41]:
temp.loc[temp.transcript_id.duplicated()].head()

,transcript_id,original_transcript_id


In [38]:
# temp2 = temp[['transcript_id', 'original_transcript_id']].merge(t_df[['transcript_id', 'original_transcript_id', 'Source']],
#                                                                 how='outer', on=['transcript_id', 'original_transcript_id'])

In [39]:
# temp2.loc[temp2.Source.isnull()]